In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import csv
from csv import reader

In [2]:
# load training dataset function
def loadData(fname):
    text=[]
    title=[]
    f= open(fname, encoding='utf8', errors="ignore")
    csv_reader = reader(f)
    
    for line in csv_reader:
        if line[0] != 'text' and line[0] != 'NA':
            job_text = line[0]
            job_text = job_text.strip().replace('\\n',' ')
            job_title = line[1]
            text.append(job_text.lower())
            title.append(job_title)
    f.close()
    return text,title

# load testing dataset function
def loadTest(fname):
    text=[]
    f= open(fname, encoding='utf8', errors="ignore")
    for line in f:   
        if len(line) > 20:
            job_text = line.strip().replace('\n',' ')
            text.append(job_text.lower())
    f.close()
    return text

In [4]:
##### train the model by the training dataset generated before
train_text,train_title = loadData('training_set.csv')
#Build a counter based on the training dataset
counter = CountVectorizer(stop_words=stopwords.words('english'))
counter.fit(train_text)

CountVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [5]:
#count the number of times each term appears in a document and transform each doc into a count vector
counts_train = counter.transform(train_text)#transform the training data
#train the classifier
model1 = DecisionTreeClassifier()
model2 = KNeighborsClassifier()
model3 = LogisticRegression(solver='liblinear')
predictors=[('nb',model1),('dt',model2),('lreg',model3)]
VT=VotingClassifier(predictors)
#train all classifier on the same datasets
VT.fit(counts_train,train_title)

VotingClassifier(estimators=[('nb', DecisionTreeClassifier()),
                             ('dt', KNeighborsClassifier()),
                             ('lreg', LogisticRegression(solver='liblinear'))])

## Notice
### load the testing data hear, could change the file name

In [6]:
#### load the testing data, could change the file name here!
test_text=loadTest('test.csv')

In [7]:
counts_test = counter.transform(test_text)#transform the testing data
#use hard voting to predict (majority voting)
pred=VT.predict(counts_test)
# convert to list
pred_title = pred.tolist()

def writecsv(outpath): 
    fw=open(outpath,'w',encoding='utf8')
    writer=csv.writer(fw,lineterminator='\n')
    writer.writerow(['text','title'])
    
    for i in range(len(test_text)):
        text = test_text[i]
        title = pred_title[i]
        #text
        writer.writerow([text,title])
    fw.close()
#### generate a new csv file including text and job title columns
outpath = 'pred_result.csv'
writecsv(outpath)

### have generated the pred_result file under the same dict